In [23]:
import sys
import xml.dom.minidom

# input file
trace_file = "trace.000.000.000.000.0.1549495911.ZltlNk.1.xml"

dom = xml.dom.minidom.parse(trace_file)  # or xml.dom.minidom.parseString(xml_string)

import pandas as pd

# convert dom into a Pandas dataframe
def dom_to_dataframe(dom, columns):
    events = dom.getElementsByTagName("Event")
    data = [dict(e.attributes.items()) for e in events] 
    return pd.DataFrame(data, columns=columns)

columns = ['As', 'ID', 'Locality', 'Machine', 'Severity', 'Transition', 'Time', 'Type']
df = dom_to_dataframe(dom, columns)


ss = df.loc[(df['Type'] == 'Role') & (df['As'] == 'StorageServer')].astype({'Severity': int, 'Time': float})
endTime =  df[-1:]['Time'].array[0]
# print endTime

ssMap = {}  # ID to a list of timestamps
nameMap = {}  # markers for this ID
for id in ss.ID.unique():
    # print id
    rows = ss.loc[ss['ID'] == id]
    for index, row in rows.iterrows():
        if id in ssMap:
            ssMap[id].append(row['Time'])
        else:
            ssMap[id] = [row['Time']]
            nameMap[id] = '{}<br>{}'.format(id, row['Machine'])

yMap = {}  # Y axis values for each ID
y = 1
for key, value in ssMap.iteritems():
    if len(value) % 2 is 1:
        value.append(endTime)
    yMap[key] = [y] * len(value)
    y += 1

In [24]:
import plotly.plotly as py
import plotly.graph_objs as go

gdata = []
annotations = []
for key, value in ssMap.iteritems():
    trace = go.Scatter(x=value, y=yMap[key], mode='lines+text+markers', text=nameMap[key], hoverinfo='x+text')
    gdata.append(trace)

    # labeling the left_side of the plot
    annotations.append(dict(xref='paper', yref='paper', x=-1, y=yMap[key][0],
                                  xanchor='right', yanchor='middle',
                                  text=key,
                                  showarrow=False))

layout = go.Layout(showlegend=False, width=1000, height=600)
fig = go.Figure(data=gdata, layout=layout)

py.iplot(fig)